<a href="https://colab.research.google.com/github/pedrohortencio/Machine-Learning-Algorithms-in-the-Real-World/blob/main/Supervised%20Learning%20Tip%20to%20Tail/svms_and_kernels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color=red>This is a draft version and the notebook is incomplete and the statements do not match the data at this point. It will be fixed soon.</font>

# SVMs and Kernels

In this notebook you will fit a Support Vector Machine (SVM) classifer to data using scikit-learn. You will use different kernels and see how kernels produce nonlinear decision surfaces. You will also predict the labels for datapoints and measure the performance of the SVM.

## Before you start

- In order for the notebooks to function as intended, modify only between lines marked "### begin your code here (__ lines)." and "### end your code here.". 

- The line count is a suggestion of how many lines of code you need to accomplish what is asked.

- You should execute the cells (the boxes that a notebook is composed of) in order.

- You can execute a cell by pressing Shift and Enter (or Return) simultaneously.

- You should have completed the previous Jupyter notebooks before attempting this one as the concepts covered there are not repeated, for the sake of brevity.

## Loading the appropriate packages

We will import SVM classifier class (SVC) as well some other packages you will use.

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import plotly.graph_objs as go

Let's turn off the scientific notation for floating point numbers.

In [ ]:
np.set_printoptions(suppress=True)

## Loading and examining the data

We will load our data from a CSV file and put it in a pandas an object of the `DataFrame` class.

This dataset is the breast cancer Wisconsin (diagnostic) dataset which contains 30 different features computed from a images of a fine needle aspirate (FNA) of breast masses for 569 patients with each example labeled as being a _benign_ or _malignant_ mass.

* This was taken and modified from the Machine Learning dataset repository of School of Information and Computer Science of University of California Irvine (UCI):
 
> _Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science._

In [ ]:
df = pd.read_csv('/content/data_svms_and_kernels.csv')

Let's take a look at the data:

In [ ]:
df.head(10)

,Feature 1,Feature 2,Label
0,0.109478,-0.168109,B
1,-0.590131,-0.153594,B
2,0.157117,0.059849,A
3,-0.623096,-0.638964,B
4,-0.296598,0.315015,B
5,-0.498929,-0.406902,B
6,0.950000,0.445315,A
7,-0.520393,-0.692691,B
8,-0.441623,-0.425413,B
9,-0.054541,-0.419501,A


To do that we first need to extract our data from the dataframe in NumPy arrays. We use `LabelEncoder` from scikit-learn to transform labels into $\{-1,+1\}$:

In [ ]:
X = df.drop('Label', axis=1).to_numpy()
y_text = df['Label'].to_numpy()
y = (2 * LabelEncoder().fit_transform(y_text)) - 1

Let's check `X`, `y_text` and `y`:

In [ ]:
X

array([[ 0.10947806, -0.16810863],
       [-0.59013059, -0.15359437],
       [ 0.15711731,  0.0598494 ],
       [-0.62309568, -0.6389643 ],
       [-0.29659811,  0.31501529],
       [-0.49892949, -0.40690183],
       [ 0.95      ,  0.44531528],
       [-0.52039318, -0.69269133],
       [-0.44162284, -0.42541306],
       [-0.05454123, -0.41950099],
       [ 0.40867744, -0.24690175],
       [ 0.00604817, -0.41695007],
       [ 0.08773186, -0.15809074],
       [-0.35794094, -0.75736888],
       [-0.06186495, -0.60203068],
       [ 0.29802828, -0.0366922 ],
       [ 0.61587189, -0.34121949],
       [-0.36402275,  0.68867891],
       [ 0.46264578, -0.43015301],
       [-0.42953288, -0.35192589],
       [ 0.20170953, -0.29872133],
       [ 0.30534941,  0.28086526],
       [-0.18209344, -0.17306767],
       [-0.09430147,  0.55499874],
       [-0.35416442, -0.32597868],
       [ 0.12331348, -0.25375275],
       [ 0.12178891, -0.44779879],
       [-0.83098402,  0.13862644],
       [ 0.18675303,

In [ ]:
X.shape

(292, 2)

Let's do the same thing for `y_text`:

In [ ]:
y_text

array(['B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'A', 'A', 'A', 'B',
       'B', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'B',
       'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'B', 'B',
       'A', 'A', 'B', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'B', 'A', 'A',
       'A', 'A', 'A', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'B', 'B',
       'B', 'A', 'A', 'A', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'A', 'B',
       'B', 'B', 'B', 'B', 'B', 'A', 'B', 'B', 'B', 'A', 'B', 'A', 'B',
       'B', 'A', 'B', 'A', 'B', 'A', 'A', 'A', 'B', 'A', 'A', 'A', 'A',
       'A', 'B', 'B', 'A', 'A', 'A', 'B', 'A', 'B', 'A', 'A', 'B', 'B',
       'B', 'A', 'B', 'B', 'B', 'B', 'A', 'A', 'B', 'B', 'A', 'A', 'B',
       'B', 'B', 'B', 'A', 'A', 'A', 'A', 'B', 'A', 'A', 'B', 'A', 'B',
       'A', 'B', 'B', 'B', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'B', 'A',
       'B', 'A', 'B', 'B', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'B',
       'B', 'B', 'B', 'A', 'B', 'B', 'B', 'B', 'A', 'B', 'A', 'B

...and for shape of `y_text`:

In [ ]:
y_text.shape

(292,)

Finally, let's check `y`:

In [ ]:
y

array([ 1,  1, -1,  1,  1,  1, -1,  1,  1, -1, -1, -1,  1,  1, -1, -1, -1,
        1, -1,  1, -1, -1,  1,  1,  1,  1, -1,  1, -1, -1,  1,  1,  1, -1,
       -1,  1,  1,  1,  1, -1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1, -1,
       -1, -1, -1, -1,  1,  1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1,
       -1,  1,  1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1,  1,  1, -1,  1,
        1,  1, -1,  1, -1,  1,  1, -1,  1, -1,  1, -1, -1, -1,  1, -1, -1,
       -1, -1, -1,  1,  1, -1, -1, -1,  1, -1,  1, -1, -1,  1,  1,  1, -1,
        1,  1,  1,  1, -1, -1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1, -1,
       -1,  1, -1, -1,  1, -1,  1, -1,  1,  1,  1,  1,  1, -1, -1, -1, -1,
       -1,  1, -1,  1, -1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,
        1,  1, -1,  1,  1,  1,  1, -1,  1, -1,  1, -1, -1,  1, -1, -1, -1,
       -1,  1, -1,  1, -1, -1,  1,  1, -1,  1, -1, -1,  1,  1, -1, -1,  1,
       -1,  1, -1, -1, -1,  1, -1,  1,  1, -1,  1,  1, -1, -1, -1, -1,  1,
        1, -1,  1, -1,  1

In [ ]:
y.shape

(292,)

Let's also do a scatter plot of our data:

In [ ]:
points_colorscale = [
                     [0.0, 'rgb(239, 85, 59)'],
                     [1.0, 'rgb(99, 110, 250)'],
                    ]

points = go.Scatter(
                    x=df['Feature 1'],
                    y=df['Feature 2'],
                    mode='markers',
                    marker=dict(color=y,
                                colorscale=points_colorscale)
                   )
layout = go.Layout(
                   xaxis=dict(range=[-1.05, 1.05]),
                   yaxis=dict(range=[-1.05, 1.05])
                  )

fig = go.Figure(data=[points], layout=layout)
fig.show()

## Splitting data

Let's split our data into training, validation and test sets. Let's use 60% for training, 20% for validation and  20% for test data.

In [ ]:
(X_train, X_vt, y_train, y_vt) = train_test_split(X, y, test_size=0.4, random_state=0)
(X_validation, X_test, y_validation, y_test) = train_test_split(X_vt, y_vt, test_size=0.5, random_state=0)

## Building and visualizing a SVM

Finally, let's build our SVM. We will use the `SVC` class from scikit-learn. For now, we are not using kernels, so we should set the `kernel` argument of `SVC` to `'linear'`. We don't need to specify any other parameters for now. You can find the documentation for `SVC` here:

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

So, make an object of class `SVC` and assign it to the name `svm`:

In [ ]:
### begin your code here (1 line).
svm = SVC(kernel='linear')
### end your code here.

Now, fit `svm` to `X_train` and `y_train`:

In [ ]:
### begin your code here (1 line).
svm.fit(X_train, y_train)
### end your code here.

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

You will get a summary for the model:

> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
>   decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
>   kernel='rbf', max_iter=-1, probability=False, random_state=None,
>   shrinking=True, tol=0.001, verbose=False)

* You may also get a warning because you have not explicitly set gamma and the default setting for that is going to change in newer versions of scikit-learn. Don't worry about that warniong.

Let's visualize the decision surface our `svm` with its supprt vectors:

In [ ]:
decision_colorscale = [
                       [0.0, 'rgb(239,  85,  59)'],
                       [0.5, 'rgb(  0,   0,   0)'],
                       [1.0, 'rgb( 99, 110, 250)']
                      ]

detail_steps = 100

(x_vis_0_min, x_vis_1_min) = (-1.05, -1.05) #X_train.min(axis=0)
(x_vis_0_max, x_vis_1_max) = ( 1.05,  1.05) #X_train.max(axis=0)

x_vis_0_range = np.linspace(x_vis_0_min, x_vis_0_max, detail_steps)
x_vis_1_range = np.linspace(x_vis_1_min, x_vis_1_max, detail_steps)

(XX_vis_0, XX_vis_1) = np.meshgrid(x_vis_0_range, x_vis_0_range)

X_vis = np.c_[XX_vis_0.reshape(-1), XX_vis_1.reshape(-1)]

YY_vis = svm.decision_function(X_vis).reshape(XX_vis_0.shape)

points = go.Scatter(
                    x=df['Feature 1'],
                    y=df['Feature 2'],
                    mode='markers',
                    marker=dict(
                                color=y,
                                colorscale=points_colorscale),
                    showlegend=False
                   )
SVs = svm.support_vectors_
support_vectors = go.Scatter(
                             x=SVs[:, 0],
                             y=SVs[:, 1],
                             mode='markers',
                             marker=dict(
                                         size=15,
                                         color='black',
                                         opacity = 0.1,
                                         colorscale=points_colorscale),
                             line=dict(dash='solid'),
                             showlegend=False
                            )

decision_surface = go.Contour(x=x_vis_0_range,
                              y=x_vis_1_range,
                              z=YY_vis,
                              contours_coloring='lines',
                              line_width=2,
                              contours=dict(
                                            start=0,
                                            end=0,
                                            size=1),
                              colorscale=decision_colorscale,
                              showscale=False
                             )

margins = go.Contour(x=x_vis_0_range,
                     y=x_vis_1_range,
                     z=YY_vis,
                     contours_coloring='lines',
                     line_width=2,
                     contours=dict(
                                   start=-1,
                                   end=1,
                                   size=2),
                     line=dict(dash='dash'),
                     colorscale=decision_colorscale,
                     showscale=False
                    )

fig2 = go.Figure(data=[margins, decision_surface, support_vectors, points], layout=layout)
fig2.show()

The datapoints, the decision surface (which is a line here), the margins and the support vectors are shown in the plot.

## Kernels

As you can see, the decision surface is underfiiting the data. Let's use a polynomial kernel. Define `svm_p2` to be an instance of class `SVC` but this time with arguments `kernel='poly'` and `degree=2` to define a degree-2 polynomial kernel:

In [ ]:
### begin your code here (1 line).
svm_p2 = SVC(kernel='poly', degree=2)
### end your code here.

..and fit it to your training data:

In [ ]:
### begin your code here (1 line).
svm_p2.fit(X_train, y_train)
### end your code here.

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=2, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

You will get a summary of your model:
    
> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
>   decision_function_shape='ovr', degree=2, gamma='auto_deprecated',
>   kernel='poly', max_iter=-1, probability=False, random_state=None,
>   shrinking=True, tol=0.001, verbose=False)

Now, let's visualize this model:

In [ ]:
YY_vis_p2 = svm_p2.decision_function(X_vis).reshape(XX_vis_0.shape)

SVs_p2 = svm_p2.support_vectors_
support_vectors_p2 = go.Scatter(
                                x=SVs_p2[:, 0],
                                y=SVs_p2[:, 1],
                                mode='markers',
                                marker=dict(
                                            size=15,
                                            color='black',
                                            opacity = 0.1,
                                            colorscale=points_colorscale),
                                line=dict(dash='solid'),
                                showlegend=False
                               )

decision_surface_p2 = go.Contour(x=x_vis_0_range,
                                 y=x_vis_1_range,
                                 z=YY_vis_p2,
                                 contours_coloring='lines',
                                 line_width=2,
                                 contours=dict(
                                               start=0,
                                               end=0,
                                               size=1),
                                 colorscale=decision_colorscale,
                                 showscale=False
                                )

margins_p2 = go.Contour(x=x_vis_0_range,
                        y=x_vis_1_range,
                        z=YY_vis_p2,
                        contours_coloring='lines',
                        line_width=2,
                        contours=dict(
                                      start=-1,
                                      end=1,
                                      size=2),
                        line=dict(dash='dash'),
                        colorscale=decision_colorscale,
                        showscale=False
                       )

fig3 = go.Figure(data=[margins_p2, decision_surface_p2, support_vectors_p2, points], layout=layout)
fig3.show()

Looks much better. But let's try a degree 3 model. Define `svm_p3` like `svm_p2` but with `degree=3` this time:

In [ ]:
### begin your code here (1 line).
svm_p3 = SVC(kernel='poly', degree=3)
### end your code here.

Next, fit your `svm_p3` model to the training data:

In [ ]:
### begin your code here (1 line).
svm_p3.fit(X_train, y_train)
### end your code here.

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Your model summary will be:

> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
>   decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
>   kernel='poly', max_iter=-1, probability=False, random_state=None,
>   shrinking=True, tol=0.001, verbose=False)

Let's visualize `svm_p3`:

In [ ]:
YY_vis_p3 = svm_p3.decision_function(X_vis).reshape(XX_vis_0.shape)

SVs_p3 = svm_p3.support_vectors_
support_vectors_p3 = go.Scatter(
                                x=SVs_p3[:, 0],
                                y=SVs_p3[:, 1],
                                mode='markers',
                                marker=dict(
                                            size=15,
                                            color='black',
                                            opacity = 0.1,
                                            colorscale=points_colorscale),
                                line=dict(dash='solid'),
                                showlegend=False
                               )

decision_surface_p3 = go.Contour(x=x_vis_0_range,
                                 y=x_vis_1_range,
                                 z=YY_vis_p3,
                                 contours_coloring='lines',
                                 line_width=2,
                                 contours=dict(
                                               start=0,
                                               end=0,
                                               size=1),
                                 colorscale=decision_colorscale,
                                 showscale=False
                                )

margins_p3 = go.Contour(x=x_vis_0_range,
                        y=x_vis_1_range,
                        z=YY_vis_p3,
                        contours_coloring='lines',
                        line_width=2,
                        contours=dict(
                                      start=-1,
                                      end=1,
                                      size=2),
                        line=dict(dash='dash'),
                        colorscale=decision_colorscale,
                        showscale=False
                       )

fig4 = go.Figure(data=[margins_p3, decision_surface_p3, support_vectors_p3, points], layout=layout)
fig4.show()

Let's try a RBF (Radial Basis Function) kernel as well. RBFs are the default kernel for scikit-learn's SVC. So build a model `svm_r` with either `kernel=rbf` argument setting or just skip the `kernel` (also the `degree` argument is uselss here, since we are not using a polynomial kernel, so just skip that): 

In [ ]:
### begin your code here (1 line).
svm_r = SVC(kernel='rbf')
### end your code here.

Fit your `svm_r` model to the training data as well:

In [ ]:
### begin your code here (1 line).
svm_r.fit(X_train, y_train)
### end your code here.

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

This will be the parameter summary:

> SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
>   decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
>   kernel='rbf', max_iter=-1, probability=False, random_state=None,
>   shrinking=True, tol=0.001, verbose=False)

We will visualize this model as well:

In [ ]:
YY_vis_r = svm_r.decision_function(X_vis).reshape(XX_vis_0.shape)

SVs_r = svm_r.support_vectors_
support_vectors_r = go.Scatter(
                                x=SVs_r[:, 0],
                                y=SVs_r[:, 1],
                                mode='markers',
                                marker=dict(
                                            size=15,
                                            color='black',
                                            opacity = 0.1,
                                            colorscale=points_colorscale),
                                line=dict(dash='solid'),
                                showlegend=False
                               )

decision_surface_r = go.Contour(x=x_vis_0_range,
                                 y=x_vis_1_range,
                                 z=YY_vis_r,
                                 contours_coloring='lines',
                                 line_width=2,
                                 contours=dict(
                                               start=0,
                                               end=0,
                                               size=1),
                                 colorscale=decision_colorscale,
                                 showscale=False
                                )

margins_r = go.Contour(x=x_vis_0_range,
                        y=x_vis_1_range,
                        z=YY_vis_r,
                        contours_coloring='lines',
                        line_width=2,
                        contours=dict(
                                      start=-1,
                                      end=1,
                                      size=2),
                        line=dict(dash='dash'),
                        colorscale=decision_colorscale,
                        showscale=False
                       )

fig5 = go.Figure(data=[margins_r, decision_surface_r, support_vectors_r, points], layout=layout)
fig5.show()

## Model selection

Let's pick the best model then. We will use the validation data for that. Let's predict using `svm` and `X_train` and assign it the name `yhat_train`. Also, predict `X_validation` and assign it the name `yhat_validation` (. The closeness of the accuracy of predictions on these two datasets will be helpful to us):

In [ ]:
### begin your code here (2 lines).
yhat_train = svm.predict(X_train)
yhat_validation = svm.predict(X_validation)
### end your code here.

Let's measure the accuracy:

In [ ]:
print('Train: ', accuracy_score(yhat_train, y_train), '\nValidation:', accuracy_score(yhat_validation, y_validation))

Train:  0.9142857142857143 
Validation: 0.9482758620689655


We got ??.??% and ??.?%.

Let's repeat thge predictions for `svm_p2` and put the results in `yhat_train_p2` and `yhat_validation_p2`:

In [ ]:
### begin your code here (2 lines).
yhat_train_p2 = svm_p2.predict(X_train)
yhat_validation_p2 = svm_p2.predict(X_validation)
### end your code here.

We can calculate the accuracies:

In [ ]:
print('Train:', accuracy_score(yhat_train_p2, y_train), '\nValidation:', accuracy_score(yhat_validation_p2, y_validation))

Train: 0.6228571428571429 
Validation: 0.5517241379310345


??.??% and ??.??%.

Let's try predicting with `svm_p3` and put it in `yhat_train_p3` and `yhat_validation_p3`:

In [ ]:
### begin your code here (2 lines).
yhat_train_p3 = svm_p3.predict(X_train)
yhat_validation_p3 = svm_p3.predict(X_validation)
### end your code here.

Now, if we predict the accuracy on these:

In [ ]:
print('Train: ', accuracy_score(yhat_train_p3, y_train), '\nValidation: ', accuracy_score(yhat_validation_p3, y_validation))

Train:  0.9085714285714286 
Validation:  0.9137931034482759


The accuracy is ??.??% and ??.??%.

Finally, let's predict `yhat_train_r` and `yhat_validatin_r` using `svm_r`:

In [ ]:
### begin your code here (2 lines).
yhat_train_r = svm_r.predict(X_train)
yhat_validation_r = svm_r.predict(X_validation)
### end your code here.

We can measure the accuracy of the SVM with RBF kernel:

In [ ]:
print('Train:', accuracy_score(yhat_train_r, y_train), '\nValidation:', accuracy_score(yhat_validation_r, y_validation))

Train: 0.9828571428571429 
Validation: 0.9827586206896551


We got ??.??% and ??.??% when using the RBF kernel.

From all these number we can see that the RBF model works best as the accuracy on validation data is high and also the gap between the accuracy on training and validation data is not big. We can further tune the generalization power of our model by tuning the argument `C` of `SVC` which is the inverse of a regularization coefficient. We won't do that here now though. 

## Final assessment

Finally, let's check accuracy on the test data to get a final performance number. Predict `yhat_test_r` from `X_test` on `svm_r`:

In [ ]:
### begin your code here (1 line).
yhat_test_r = svm_r.predict(X_test)
### end your code here.
accuracy_score(yhat_test_r, y_test)

0.9661016949152542

??.??%. We have good performance of test data.